In [ ]:
import modulos as md
import numpy as np
import constantes as cons
import os
import sys
lista_filtrada = []

lista=md.lista_de_monedas()
#lista=md.filtradodemonedas ()
#lista=['SPELLUSDT', 'AXSUSDT', 'ETHUSDT', 'BNBUSDT', 'OGNUSDT', 'CYBERUSDT', '1000PEPEUSDT', 'COMPUSDT', 'ADAUSDT', 'BCHUSDT', 'RUNEUSDT', 'MATICUSDT', 'BLZUSDT', 'HBARUSDT', 'TRBUSDT', 'WLDUSDT', 'MTLUSDT', 'BTCUSDT', 'XRPUSDT', 'OPUSDT', 'PERPUSDT', 'ARBUSDT', 'STMXUSDT', 'CFXUSDT', 'SOLUSDT', 'DOGEUSDT', 'APEUSDT', 'UNFIUSDT', 'LTCUSDT', 'STORJUSDT', 'CRVUSDT']
#lista=['STMXUSDT']

if len(lista)>1:
    imprimo=False
    debug=False
else:
    imprimo= True
    debug=True
for symbol in lista:    
    try:
        #data,porcentajeentrada = md.estrategia_santa(symbol,tp_flag = True)
        #data,porcentajeentrada = md.estrategia_trampa(symbol,tp_flag = True)
        data = md.estrategia_haz(symbol, debug=debug)
        #data,porcentajeentrada = md.estrategia_oro('XAU',tp_flag = True)
        #######################################################################
        resultado = md.backtesting(data, plot_flag = imprimo)
        #resultado = md.backtestingsanta(data, plot_flag = imprimo, debug = True)
        ########################################################################
        #if resultado['Return [%]'] < 0: # para santa3 mazmorra
        #if data.disparo.iloc[-1]!=0: # imprime las monedas que estan en posible haz martillo
        #        print(symbol)
        if resultado['Return [%]'] > 0:
            #if ((resultado['Profit Factor'] > 2 or np.isnan(resultado['Profit Factor'])) and (resultado['Return [%]']/resultado['# Trades']) >=0.33):            
                lista_filtrada.append(symbol)
        print(f"{symbol} - Return [%]: {md.truncate(resultado['Return [%]'],2)}% - # Trades: {resultado['# Trades']} - Profit Factor: {resultado['Profit Factor']} - Win Rate [%]: {resultado['Win Rate [%]']}")
    except Exception as falla:
        _, _, exc_tb = sys.exc_info()
        fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
        print("\nError: "+str(falla)+" - line: "+str(exc_tb.tb_lineno)+" - file: "+str(fname)+"\n")
        pass
for symbol in lista_filtrada:
    md.printandlog(cons.nombrelog,symbol,pal=1)
#md.dibuja_patrones_triangulos (data,998)    


In [1]:
import modulos as md
import numpy as np
from backtesting import Backtest
import pandas_ta as ta
from backtesting import Strategy

def backtesting(data, plot_flag=False):
    balance = 100    
    def ema(data):
        indi=ta.ema(data.Close.s,length=21)
        return indi.to_numpy()
  
    class Fenix(Strategy):
        def init(self):
            super().init()
            self.ema = self.I(ema,self.data)
            
        def next(self):       
            super().next()
            if self.position:
                if self.data.cierra[-1]==True:
                    self.position.close()                    
            else:   
                if np.isnan(data.take_profit[-1]):
                    tp_value = None
                else:
                    tp_value = self.data.take_profit[-1]
                size= balance*self.data.porcentajeentrada[-1]/100
                if self.data.signal[-1]==1:
                    self.buy(size=size,sl=self.data.stop_loss[-1],tp=tp_value)
                elif self.data.signal[-1]==-1:
                    self.sell(size=size,sl=self.data.stop_loss[-1],tp=tp_value)
    bt = Backtest(data, Fenix, cash=balance)
    output = bt.run()
    if plot_flag:
        bt.plot()
    return output

####################################################################################

def detect_pivot_points(df):
    # Función para detectar los puntos pivote High y Low
    # Parámetros de longitud para los puntos pivote High y Low
    leftLenH = 10
    rightLenH = 10
    leftLenL = 10
    rightLenL = 10
    df['pivot_high'] = np.NaN
    df['pivot_low'] = np.NaN
    df['row_number'] = (range(len(df)))
    df.set_index('row_number', inplace=True)
    for i in range(leftLenH, len(df) - rightLenH):
        if (
            df['High'].iloc[i] == df['High'].iloc[i - leftLenH:i + rightLenH + 1].max()
            and df['High'].iloc[i] > df['High'].iloc[i - 1]
            and df['High'].iloc[i] > df['High'].iloc[i + 1]
            ):
            df.at[i, 'pivot_high'] = df['High'].iloc[i]
    for i in range(leftLenL, len(df) - rightLenL):
        if (
            df['Low'].iloc[i] == df['Low'].iloc[i - leftLenL:i + rightLenL + 1].min()
            and df['Low'].iloc[i] < df['Low'].iloc[i - 1]
            and df['Low'].iloc[i] < df['Low'].iloc[i + 1]
            ):
            df.at[i, 'pivot_low'] = df['Low'].iloc[i]
    df['timestamp']=df['Open Time']
    df.set_index('timestamp', inplace=True)

##########################################################################################

def estrategia_real(symbol,debug = False):
    try:                
        np.seterr(divide='ignore', invalid='ignore')
        # temporalidad de 1h para encontrar martillos y soportes/resistencias
        data = md.obtiene_historial(symbol,'1h')      
        detect_pivot_points(data)  
        data['signal'] = 0
        data['take_profit'] = np.NaN
        data['stop_loss'] = np.NaN
        data['cierra'] = False
        # Reemplazar valores no finitos (NA e inf) con 0
        data['porcentajeentrada'] = np.nan_to_num((data.Close/data.atr), nan=0, posinf=0, neginf=0)
        # Aplicar np.floor y convertir a enteros
        data['porcentajeentrada'] = np.floor(data['porcentajeentrada']).astype(int)
        ####################### alertas y valores
        if debug:
            df_str = data[list(data.columns)].to_string(index=False)
            print(df_str)
        return data
    except Exception as falla:
        _, _, exc_tb = sys.exc_info()
        fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
        print("\nError: "+str(falla)+" - line: "+str(exc_tb.tb_lineno)+" - file: "+str(fname)+" - symbol: "+symbol+"\n")
        pass

#######################################################################################################################


symbol='ATOMUSDT'
df = estrategia_real(symbol)

#for index, values in df.iterrows():
#    if values.PivotLow ==1 or values.PivotHigh ==1:
#        print(f"{values['Open Time']} - {values.PivotLow} - {values.PivotHigh}")

df.tail(60)     

,Open Time,Open,High,Low,Close,Volume,ema20,ema50,ema200,atr,pivot_high,pivot_low,signal,take_profit,stop_loss,cierra,porcentajeentrada
timestamp,,,,,,,,,,,,,,,,,
2023-09-14 10:00:00,2023-09-14 10:00:00,6.545,6.585,6.537,6.584,223772.83,6.534056,6.510874,6.639154,0.045803,NaN,NaN,0,NaN,NaN,False,143
2023-09-14 11:00:00,2023-09-14 11:00:00,6.583,6.588,6.565,6.584,164092.79,6.538813,6.513741,6.638605,0.044174,NaN,NaN,0,NaN,NaN,False,149
2023-09-14 12:00:00,2023-09-14 12:00:00,6.584,6.605,6.552,6.602,324965.33,6.544830,6.517202,6.638241,0.044804,NaN,NaN,0,NaN,NaN,False,147
2023-09-14 13:00:00,2023-09-14 13:00:00,6.602,6.657,6.595,6.614,523806.16,6.551418,6.520998,6.638000,0.046032,NaN,NaN,0,NaN,NaN,False,143
2023-09-14 14:00:00,2023-09-14 14:00:00,6.615,6.663,6.598,6.652,296064.75,6.560997,6.526136,6.638139,0.047387,NaN,NaN,0,NaN,NaN,False,140
2023-09-14 15:00:00,2023-09-14 15:00:00,6.653,6.677,6.632,6.654,396453.41,6.569855,6.531150,6.638297,0.047217,NaN,NaN,0,NaN,NaN,False,140
2023-09-14 16:00:00,2023-09-14 16:00:00,6.655,6.680,6.624,6.679,288378.95,6.580249,6.536948,6.638702,0.047844,NaN,NaN,0,NaN,NaN,False,139
2023-09-14 17:00:00,2023-09-14 17:00:00,6.679,6.708,6.661,6.699,335703.48,6.591559,6.543303,6.639302,0.047784,NaN,NaN,0,NaN,NaN,False,140
2023-09-14 18:00:00,2023-09-14 18:00:00,6.700,6.819,6.699,6.705,924015.44,6.602363,6.549644,6.639955,0.052942,NaN,NaN,0,NaN,NaN,False,126
